In [1]:
import pandas as pd
import numpy as np

# Path to your data file
path = "/Users/prithvivarman/Desktop/Projects/TWSQ DataProject 2/DataProject.pk"   # change if needed

# Load the pickled DataFrame
data = pd.read_pickle(path)

print("Raw shape:", data.shape)
print(data.head())


Raw shape: (3979, 4)
         date    asset        value        item
0  2020-01-01   ADAUSD     0.033813  daily_high
1  2020-01-01   BTCUSD  7254.330566  daily_high
2  2020-01-01   ETHUSD   132.835358  daily_high
3  2020-01-01  LINKUSD     1.851417  daily_high
4  2020-01-02   ADAUSD     0.033507  daily_high


In [3]:
# --- 1. Basic typing & sorting ---
data["date"] = pd.to_datetime(data["date"])
data = data.sort_values(["date", "asset", "item"])

print("\nUnique items:", data["item"].unique())
print("Unique assets (raw):", sorted(data["asset"].unique()))

# --- 2. Normalise asset names (BTCUSD -> BTC-USD, etc.) ---
rename_map = {
    "ADAUSD": "ADA-USD",
    "BTCUSD": "BTC-USD",
    "ETHUSD": "ETH-USD",
    "LINKUSD": "LINK-USD",
}
data["asset"] = data["asset"].replace(rename_map)

print("Unique assets (after rename):", sorted(data["asset"].unique()))

# --- 3. Convert sentinel -999.0 to NaN (missing) ---
# The dataset uses -999.0 as a placeholder for "no return / missing"
data.loc[data["value"] == -999.0, "value"] = np.nan

# --- 4. Pivot into wide DataFrames: one for each item ---
high = (data[data["item"] == "daily_high"]
        .pivot(index="date", columns="asset", values="value")
        .sort_index())

low = (data[data["item"] == "daily_low"]
       .pivot(index="date", columns="asset", values="value")
       .sort_index())

ret = (data[data["item"] == "daily_ret"]
       .pivot(index="date", columns="asset", values="value")
       .sort_index())

print("\nPivot shapes: high", high.shape, "low", low.shape, "ret", ret.shape)
print("\nHead of returns:")
print(ret.head())

# --- 5. High/low sanity check: high should never be below low ---
common_hl_index = high.index.intersection(low.index)

hl_problem = (high.loc[common_hl_index] < low.loc[common_hl_index])
num_problems = hl_problem.sum().sum()
print("\nNumber of high<low problems:", num_problems)

if num_problems > 0:
    prob_locs = hl_problem.stack()
    prob_locs = prob_locs[prob_locs]
    print("Problem locations (date, asset):")
    print(prob_locs)

    # Simple fix: set high equal to low for those cells
    for (dt, asset) in prob_locs.index:
        high.loc[dt, asset] = low.loc[dt, asset]

# --- 6. Clean return matrix: drop rows with ANY NaN returns ---
# (simple first pass; later we could be fancier)
ret_clean = ret.dropna(how="any")
print("\nReturn matrix after dropna, shape:", ret_clean.shape)
print(ret_clean.head())



Unique items: ['daily_low' 'daily_high' 'daily_ret']
Unique assets (raw): ['ADA-USD', 'ADAUSD', 'BTC-USD', 'BTCUSD', 'ETH-USD', 'ETHUSD', 'LINK-USD', 'LINKUSD']
Unique assets (after rename): ['ADA-USD', 'BTC-USD', 'ETH-USD', 'LINK-USD']

Pivot shapes: high (362, 4) low (271, 4) ret (362, 4)

Head of returns:
asset        ADA-USD  BTC-USD  ETH-USD  LINK-USD
date                                            
2020-01-01       NaN      NaN      NaN       NaN
2020-01-02 -0.021131      NaN      NaN       NaN
2020-01-03  0.043632      NaN      NaN       NaN
2020-01-04  0.012142      NaN      NaN       NaN
2020-01-05  0.003642      NaN      NaN       NaN

Number of high<low problems: 1
Problem locations (date, asset):
date        asset  
2020-03-15  BTC-USD    True
dtype: bool

Return matrix after dropna, shape: (241, 4)
asset        ADA-USD   BTC-USD   ETH-USD  LINK-USD
date                                              
2020-05-02 -0.004799  0.013969  0.005164  0.010308
2020-05-03 -0.040871 -0

In [5]:
# We’ll use a 5-day lookback for cross-sectional momentum
lookback = 5

# Momentum signal = rolling 5-day sum of returns
mom = ret_clean.rolling(window=lookback).sum()

print("\nMomentum signal (first 10 rows):")
print(mom.head(10))



Momentum signal (first 10 rows):
asset        ADA-USD   BTC-USD   ETH-USD  LINK-USD
date                                              
2020-05-02       NaN       NaN       NaN       NaN
2020-05-03       NaN       NaN       NaN       NaN
2020-05-04       NaN       NaN       NaN       NaN
2020-05-05       NaN       NaN       NaN       NaN
2020-05-06 -0.028523  0.045193 -0.048186 -0.038753
2020-05-07  0.003181  0.104887 -0.013000 -0.016733
2020-05-08  0.067474  0.104086  0.010706  0.027572
2020-05-09  0.041612  0.077105  0.017254  0.072378
2020-05-10 -0.030683 -0.020331 -0.084721  0.024612
2020-05-11 -0.029929 -0.067502 -0.085819 -0.008356


In [7]:
# Initialise weights with zeros (same shape as mom)
weights = pd.DataFrame(index=mom.index,
                       columns=mom.columns,
                       data=0.0)

for dt in mom.index:
    sig = mom.loc[dt]

    # If all NaN (early days) skip
    if sig.isna().all():
        continue

    # drop NaNs if any
    sig = sig.dropna()
    if sig.empty:
        continue

    # Find best and worst coin for that day
    best_asset = sig.idxmax()
    worst_asset = sig.idxmin()

    # Long +0.5 in best, short -0.5 in worst
    weights.loc[dt, best_asset]  =  0.5
    weights.loc[dt, worst_asset] = -0.5

# Show some non-zero weight rows
print("\nNon-zero weights sample:")
print(weights[(weights != 0).any(axis=1)].head())



Non-zero weights sample:
asset       ADA-USD  BTC-USD  ETH-USD  LINK-USD
date                                           
2020-05-06      0.0      0.5     -0.5       0.0
2020-05-07      0.0      0.5      0.0      -0.5
2020-05-08      0.0      0.5     -0.5       0.0
2020-05-09      0.0      0.5     -0.5       0.0
2020-05-10      0.0      0.0     -0.5       0.5


In [9]:
# Shift weights by one day (today's return uses yesterday's signal)
weights_lag = weights.shift(1)

# Align returns with weight index (should already match)
ret_aligned = ret_clean.reindex(weights_lag.index)

# Portfolio daily return = sum over assets of w_{t-1} * r_t
port_ret = (weights_lag * ret_aligned).sum(axis=1)

print("\nPortfolio return series (first 10 non-NaN days):")
print(port_ret.dropna().head(10))



Portfolio return series (first 10 non-NaN days):
date
2020-05-02    0.000000
2020-05-03    0.000000
2020-05-04    0.000000
2020-05-05    0.000000
2020-05-06    0.000000
2020-05-07    0.016656
2020-05-08   -0.014538
2020-05-09   -0.009371
2020-05-10    0.010703
2020-05-11   -0.016308
dtype: float64


In [11]:
# Drop initial NaNs (before we had enough history)
pr = port_ret.dropna()
n = len(pr)

# --- 5.1 Mean & volatility (daily) ---
mean_daily = pr.mean()
vol_daily  = pr.std(ddof=1)     # sample stdev

sharpe_daily = mean_daily / vol_daily if vol_daily != 0 else np.nan

# Annualised versions (252 trading days)
mean_annual   = mean_daily * 252
vol_annual    = vol_daily * np.sqrt(252)
sharpe_annual = sharpe_daily * np.sqrt(252)

# --- 5.2 t-stat for "mean > 0" ---
# t = mean / (sd / sqrt(n))
t_stat = mean_daily / (vol_daily / np.sqrt(n)) if vol_daily != 0 else np.nan

print("\nPerformance statistics:")
print("Number of days         :", n)
print("Mean daily return      :", mean_daily)
print("Daily volatility       :", vol_daily)
print("Daily Sharpe           :", sharpe_daily)
print("Annualised mean return :", mean_annual)
print("Annualised volatility  :", vol_annual)
print("Annualised Sharpe      :", sharpe_annual)
print("t-stat (mean > 0)      :", t_stat)

# --- 5.3 Drawdown and drawdown duration ---

# Equity curve starting from 1
equity = (1 + pr).cumprod()

# Running peak of equity
running_max = equity.cummax()

# Drawdown series: current / peak - 1 (negative numbers)
drawdown = equity / running_max - 1

max_dd = drawdown.min()   # worst drawdown (most negative)

# Drawdown duration: longest stretch underwater
underwater = equity < running_max
max_duration = 0
current = 0
for flag in underwater:
    if flag:
        current += 1
        if current > max_duration:
            max_duration = current
    else:
        current = 0

print("\nRisk statistics:")
print("Max drawdown                :", max_dd)
print("Max drawdown duration (days):", max_duration)



Performance statistics:
Number of days         : 241
Mean daily return      : 0.002472451814149816
Daily volatility       : 0.02624095691011803
Daily Sharpe           : 0.09422109958179477
Annualised mean return : 0.6230578571657537
Annualised volatility  : 0.41656227689120523
Annualised Sharpe      : 1.4957135864908853
t-stat (mean > 0)      : 1.462704809981494

Risk statistics:
Max drawdown                : -0.32035988832123696
Max drawdown duration (days): 135


In [15]:
# =============================================================================
# STEP 6: reusable cross-sectional momentum backtest function
# =============================================================================

import numpy as np
import pandas as pd

def backtest_cs_mom(ret_df, lookback=5, n_long=1, n_short=1):
    """
    Cross-sectional momentum backtest.

    ret_df : DataFrame of daily returns (rows: dates, cols: assets)
    lookback : number of days used to compute momentum (rolling sum)
    n_long : how many top assets to long each day
    n_short: how many bottom assets to short each day
    """
    # 1. Momentum signal (rolling sum of past 'lookback' days)
    mom = ret_df.rolling(window=lookback).sum()

    # 2. Build weights (initially all zeros)
    weights = pd.DataFrame(index=mom.index,
                           columns=mom.columns,
                           data=0.0)

    for dt in mom.index:
        sig = mom.loc[dt]

        # skip days where signal is all NaN (early period)
        if sig.isna().all():
            continue

        sig = sig.dropna()
        if len(sig) == 0:
            continue

        # sort descending: winners at top, losers at bottom
        sig_sorted = sig.sort_values(ascending=False)

        # choose which assets to long/short
        long_assets  = sig_sorted.index[:n_long]
        short_assets = sig_sorted.index[-n_short:]

        # equal-weight long and short so total long = +0.5, total short = -0.5
        long_weight  =  0.5 / n_long if n_long > 0 else 0.0
        short_weight = -0.5 / n_short if n_short > 0 else 0.0

        for a in long_assets:
            weights.loc[dt, a] = long_weight
        for a in short_assets:
            weights.loc[dt, a] = short_weight

    # 3. Lag weights by 1 day to avoid look-ahead
    w_lag = weights.shift(1)

    # 4. Align returns
    r = ret_df.reindex(w_lag.index)

    # 5. Portfolio return series
    port_ret = (w_lag * r).sum(axis=1).dropna()

    # 6. Performance & risk stats (same style as before)
    n = len(port_ret)
    mean_daily = port_ret.mean()
    vol_daily  = port_ret.std(ddof=1)
    sharpe_daily = mean_daily / vol_daily if vol_daily != 0 else np.nan

    mean_annual   = mean_daily * 252
    vol_annual    = vol_daily * np.sqrt(252)
    sharpe_annual = sharpe_daily * np.sqrt(252)

    t_stat = mean_daily / (vol_daily / np.sqrt(n)) if vol_daily != 0 else np.nan

    # Drawdown
    equity = (1 + port_ret).cumprod()
    running_max = equity.cummax()
    drawdown = equity / running_max - 1
    max_dd = drawdown.min()

    underwater = equity < running_max
    max_duration = 0
    current = 0
    for flag in underwater:
        if flag:
            current += 1
            if current > max_duration:
                max_duration = current
        else:
            current = 0

    stats = {
        "lookback": lookback,
        "n_long": n_long,
        "n_short": n_short,
        "n_days": n,
        "mean_daily": mean_daily,
        "vol_daily": vol_daily,
        "sharpe_daily": sharpe_daily,
        "mean_annual": mean_annual,
        "vol_annual": vol_annual,
        "sharpe_annual": sharpe_annual,
        "t_stat": t_stat,
        "max_dd": max_dd,
        "max_dd_days": max_duration,
    }

    return port_ret, stats


In [17]:
# =============================================================================
# STEP 7: reproduce original strategy (5-day momentum, long 1 / short 1)
# =============================================================================

port_ret_base, stats_base = backtest_cs_mom(
    ret_clean,
    lookback=5,   # same 5-day momentum as before
    n_long=1,     # long best 1 coin
    n_short=1     # short worst 1 coin
)

print("\n=== BASE STRATEGY (5-day, 1 long / 1 short) ===")
for k, v in stats_base.items():
    print(f"{k:15s}: {v}")



=== BASE STRATEGY (5-day, 1 long / 1 short) ===
lookback       : 5
n_long         : 1
n_short        : 1
n_days         : 241
mean_daily     : 0.002472451814149816
vol_daily      : 0.02624095691011803
sharpe_daily   : 0.09422109958179477
mean_annual    : 0.6230578571657537
vol_annual     : 0.41656227689120523
sharpe_annual  : 1.4957135864908853
t_stat         : 1.462704809981494
max_dd         : -0.32035988832123696
max_dd_days    : 135


In [19]:
# =============================================================================
# STEP 8: use more breadth (5-day, long 2 / short 2)
# =============================================================================

port_ret_5_2_2, stats_5_2_2 = backtest_cs_mom(
    ret_clean,
    lookback=5,
    n_long=2,
    n_short=2
)

print("\n=== STRATEGY: 5-day, 2 long / 2 short ===")
for k, v in stats_5_2_2.items():
    print(f"{k:15s}: {v}")



=== STRATEGY: 5-day, 2 long / 2 short ===
lookback       : 5
n_long         : 2
n_short        : 2
n_days         : 241
mean_daily     : 0.001419945875277547
vol_daily      : 0.01684330973628942
sharpe_daily   : 0.0843032573472321
mean_annual    : 0.3578263605699418
vol_annual     : 0.2673792529047283
sharpe_annual  : 1.3382727219207295
t_stat         : 1.3087384945221978
max_dd         : -0.13603910104534256
max_dd_days    : 129


In [21]:
# =============================================================================
# STEP 9: try different lookbacks and long/short counts
# =============================================================================

results = []

for lookback in [3, 5, 10, 20]:
    for (n_long, n_short) in [(1, 1), (2, 2)]:
        _, stats = backtest_cs_mom(
            ret_clean,
            lookback=lookback,
            n_long=n_long,
            n_short=n_short
        )
        results.append(stats)

res_df = pd.DataFrame(results)
res_df = res_df[[
    "lookback", "n_long", "n_short",
    "sharpe_annual", "mean_annual", "vol_annual",
    "max_dd", "max_dd_days"
]]

print("\n=== PARAMETER SWEEP RESULTS ===")
print(res_df.sort_values("sharpe_annual", ascending=False))



=== PARAMETER SWEEP RESULTS ===
   lookback  n_long  n_short  sharpe_annual  mean_annual  vol_annual  \
7        20       2        2       2.345189     0.664559    0.283371   
1         3       2        2       2.290497     0.647476    0.282679   
6        20       1        1       1.674697     0.691124    0.412686   
5        10       2        2       1.565686     0.429681    0.274436   
2         5       1        1       1.495714     0.623058    0.416562   
0         3       1        1       1.393591     0.599774    0.430380   
3         5       2        2       1.338273     0.357826    0.267379   
4        10       1        1       1.056859     0.450492    0.426255   

     max_dd  max_dd_days  
7 -0.166545           39  
1 -0.121442           47  
6 -0.202586          112  
5 -0.193409           96  
2 -0.320360          135  
0 -0.227136           96  
3 -0.136039          129  
4 -0.355063          135  


In [23]:
# =============================================================================
# STEP 10: final chosen strategy (20-day, 2 long / 2 short)
# =============================================================================

lookback_best = 20
n_long_best   = 2
n_short_best  = 2

port_ret_best, stats_best = backtest_cs_mom(
    ret_clean,
    lookback=lookback_best,
    n_long=n_long_best,
    n_short=n_short_best
)

print("\n=== FINAL CHOSEN STRATEGY ===")
for k, v in stats_best.items():
    print(f"{k:15s}: {v}")



=== FINAL CHOSEN STRATEGY ===
lookback       : 20
n_long         : 2
n_short        : 2
n_days         : 241
mean_daily     : 0.002637139752781511
vol_daily      : 0.017850714407847607
sharpe_daily   : 0.1477330090286
mean_annual    : 0.6645592177009407
vol_annual     : 0.2833713062880143
sharpe_annual  : 2.345188813949613
t_stat         : 2.293433040564146
max_dd         : -0.16654467902654324
max_dd_days    : 39
